In [78]:
# Connecting to the Database in mySQL

import mysql.connector
from mysql.connector import Error
from decimal import Decimal

try:
    connection = mysql.connector.connect(
        host='localhost',           # Hostname (assuming a local database)
        user='root',       # Your MySQL username (also assuming local database)
        password='YourPassword',   # Your MySQL password (please don't steal if I forgot to erase mine lol)
        database='ECOMMERCE'
    )

    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as error:
    print("Error while connecting to MySQL", error)



Connected to MySQL Server version  8.3.0
You're connected to database:  ('ecommerce',)


In [79]:
# Trigger #1
# This will update the inventory values for products after transactions have happened

createTrigger= """
CREATE TRIGGER UpdateInventoryForPlacedOrders
AFTER INSERT ON TRANSACTION_DETAIL
FOR EACH ROW
BEGIN
    DECLARE inventory_count INT;
    SELECT Available 
    INTO inventory_count 
    FROM INVENTORY WHERE 
    ProductID = NEW.ProductID;
    IF inventory_count IS NOT NULL THEN
        UPDATE INVENTORY 
        SET Available = inventory_count - NEW.Quantity 
        WHERE ProductID = NEW.ProductID;
    END IF;
END;
"""

try: 
    cursor.execute(createTrigger)
    connection.commit()
    print("Trigger 'UpdateInventoryForPlacedOrders' Successfully created!")
except mysql.connector.Error as error:
    print("Error creating trigger:", error)
    


Error creating trigger: 1359 (HY000): Trigger already exists


In [121]:
# This will get the list of inventory products
InventoryListQuery = """
SELECT * 
FROM INVENTORY;
"""

def InventoryList():
    try:
        cursor.execute(InventoryListQuery)
        
        print("Inventory: ")
        print(f"{'InventoryID':<15} {'ProductID':<15} {'Available':<15}")
        for product in cursor: 
            print(f"{product[0]:<15} {product[1]:<15} {product[2]:<15}")
        
        connection.commit()
    except mysql.connector.Error as error: 
        print("Error fetching inventory: ", error)
        connection.rollback()


In [81]:
# Procedure #1
# This will Add products to the product an dinventory tables
createProductProcedure = """
CREATE PROCEDURE AddProductAndInventory(
    IN product_name VARCHAR(100),
    IN description MEDIUMTEXT,
    IN price DECIMAL(15,2),
    IN initial_quantity INT,
    IN product_type VARCHAR(20),
    IN battery_type VARCHAR(100),
    IN collectible_condition VARCHAR(100)
)
BEGIN
    DECLARE product_id INT;

    INSERT INTO PRODUCT (Product_name, Description, Price)
    VALUES (product_name, description, price);

    SET product_id = LAST_INSERT_ID();

    INSERT INTO INVENTORY (ProductID, Available)
    VALUES (product_id, initial_quantity);

    IF product_type = 'electronic' THEN
        INSERT INTO ELECTRONIC (ProductID, BatteryType)
        VALUES (product_id, battery_type);
    ELSEIF product_type = 'collectible' THEN
        INSERT INTO COLLECTIBLE (ProductID, CollectibleCondition)
        VALUES (product_id, collectible_condition);
    END IF;
END
"""
try: 
    cursor.execute(createProductProcedure)
    print("Created AddProductAndInventory Procedure Succefully")
except mysql.connector.Error as error: 
    print("Error when trying to make Procedure: ", error)

def createProduct():
    
    try: 
        ProductName = input("What is the Products Name? ")
        description = input("Description of product (leave empty if not needed.): ")
        Quantity = input("How many Products are you adding? ")
        price = input("What is the Price of the Product? ")
        productType = input("What is the product's Type Electronic/Collectible? (Leave empty if neither.)")
        productType = productType.lower()
        batteryType = None
        collectibleCondition = None
        
        if productType == 'electronic':
            batteryType = input("What is the battery type? ")
        elif productType == 'collectible':
            collectibleCondition = input("What is the collectible's condition?")
    
        cursor.callproc("AddProductAndInventory", (ProductName, description, price, Quantity, productType, batteryType, collectibleCondition))
        connection.commit()
        print("Added Product Succesfully")
    except mysql.connector.Error as error: 
        print("Error when trying to add product. Aborting:  ", error)
    
    

Error when trying to make Procedure:  1304 (42000): PROCEDURE AddProductAndInventory already exists


In [84]:
# Procedure #2
# This will edit the inventory that we currently have
editInvProcedure = """
CREATE PROCEDURE UpdateInventory(
    IN updateProductId INT,
    IN quantityChange INT
)
BEGIN
    UPDATE INVENTORY
    SET Available = Available + quantityChange
    WHERE ProductID = updateProductId;
END
"""

try:
    cursor.execute(editInvProcedure)
    print("Stored procedure 'UpdateInventory' created successfully.")
except mysql.connector.Error as error:
    print("Error creating stored procedure:", error)
    

def editInventoryProd():
    try:
        updateProductId = int(input("Enter the Product ID: "))
        change = int(input("Enter the quantity change (+ for increase, - for decrease): "))
        cursor.callproc("UpdateInventory", (updateProductId, change))
        connection.commit()
        print("Inventory updated successfully!")
    except ValueError:
        print("Invalid input. Please enter valid Product ID and quantity change.")
    except mysql.connector.Error as error:
        print("Error updating inventory:", error)

    

Error creating stored procedure: 1304 (42000): PROCEDURE UpdateInventory already exists


In [85]:
#Deletes the product 30003 from the table 
delQueryProduct = """
DELETE FROM PRODUCT
WHERE ProductID= %s;
"""
delQueryInventory = """
DELETE FROM INVENTORY
WHERE ProductID = %s;
"""

def deleteProd():
    try:
        userProductID = int(input("Enter the ProductID: "))
        cursor.execute(delQueryProduct, (userProductID,))
        cursor.execute(delQueryInventory, (userProductID,))
        print("Product has been successfully deleted from Product table")
    except mysql.connector.Error as error: 
        print("Error when trying delete Product: ", error)


In [86]:
#Joins both the review and Transacation detail tables to give most popular products based on quantity sold and ratings after 2024.
mostPopQuery="""
SELECT REVIEW.ProductID, SUM(Quantity) as Total, Rating
FROM REVIEW
RIGHT JOIN TRANSACTION_DETAIL
ON REVIEW.ProductID = TRANSACTION_DETAIL.ProductID
WHERE RatingDate > '2024-1-1'
GROUP BY TRANSACTION_DETAIL.ProductID, Rating
ORDER BY Total DESC, Rating DESC;
"""

def getMostPop():
    try:
        cursor.execute(mostPopQuery)
        print("Most Popular items: ")
        print("tProductID\tQuantitySold\tRating")
        for columns in cursor: 
            print(f"{columns[0]}\t\t{columns[1]}\t\t{columns[2]}")
        connection.commit()
    except mysql.connector.Error as error: 
        print("Error fetching inventory: ", error)
        connection.rollback()

In [87]:
#Joins both the review and Transacation detail tables to give least popular products based on quantity sold and ratings after 2024.
leastPopQuery= """
SELECT REVIEW.ProductID, SUM(Quantity) as Total, Rating
FROM REVIEW
RIGHT JOIN TRANSACTION_DETAIL
ON REVIEW.ProductID = TRANSACTION_DETAIL.ProductID
WHERE RatingDate > '2024-1-1'
GROUP BY TRANSACTION_DETAIL.ProductID, Rating
ORDER BY Total, Rating;
"""

def getLeastPop():
    try:
        cursor.execute(leastPopQuery)
        print("Least Popular items: ")
        print("ProductID\tQuantitySold\tRating")
        for columns in cursor: 
            print(f"{columns[0]}\t\t{columns[1]}\t\t{columns[2]}")
        connection.commit()
    except mysql.connector.Error as error: 
        print("Error fetching inventory: ", error)
        connection.rollback()

In [88]:
# This will get Users that have not have anything in their Order History from the Last three months
AfkQuery = """
SELECT DISTINCT UserID, UserName, Email
FROM USERS
WHERE UserID NOT IN(
    SELECT DISTINCT UserID
    FROM ORDER_HISTORY
    WHERE OrderDate >= DATE_SUB(NOW(), INTERVAL 3 MONTH)
)
"""

def getAfkUsers():
    try:
        cursor.execute(AfkQuery)
        print("Users to send Promotional Emails to. ")
        print("UserID\t\tUserName\t\t\tEmail")
        for users in cursor:
            print(f"{users[0]}\t\t{users[1]}\t\t\t{users[2]}")
            
        connection.commit()
    except mysql.connector.Error as error:
        print("Error when trying to find Users to send Promos to :", error)
        connection.rollback()

In [89]:
#This will create a new transaction

def createNewTransaction():
    try: 
        OrderTotal = 0
        userID = int(input("What is the Users ID? "))
        orderDate = input("Enter the Order Date (YYYY-MM-DD):" )
        orderStatus = input("What is the Order Status? (Delivered, Cancelled, Processing, Shipped)")
        DistinctProductCount = int(input("How many distinct products are you buying? "))
        distintProducts = []
        
        for index in range(DistinctProductCount):
            productid = int(input(f" Product ID for product {index + 1}: "))
            Quantity = int(input(f"Quantity for product {index+1}: "))
            distintProducts.append((productid, Quantity))
        
        for productid, quantity in distintProducts:
            cursor.execute(f"""
                           SELECT Price
                           FROM PRODUCT
                           WHERE ProductID = {productid}
                           """)
            price = cursor.fetchone()[0]
            if isinstance(price, Decimal):
                purchaseTotal = price * quantity
            else:
                price = Decimal(price)
                purchaseTotal = price * Quantity
            OrderTotal += purchaseTotal
            
        orderQuery = """
            INSERT INTO ORDER_HISTORY (UserID, OrderDate, OrderStatus, OrderTotal)
            VALUES (%s, %s, %s, %s)
            """
        orderValues = (userID, orderDate, orderStatus, OrderTotal)
        cursor.execute(orderQuery, orderValues)
        orderID = cursor.lastrowid
        
        transactionQuery = """
            INSERT INTO TRANSACTIONS (OrderID, UserID, PurchaseAmount, TransactionDate)
            VALUES (%s, %s, %s, %s)
            """
        transactionValues = (orderID, userID, OrderTotal, orderDate)
        cursor.execute(transactionQuery, transactionValues)
        transactionID = cursor.lastrowid
        
        for productid, Quantity in distintProducts:
            cursor.execute(f"""
                    SELECT Price
                    FROM PRODUCT
                    WHERE ProductID = {productid}
                """)
            price = cursor.fetchone()[0]
            if isinstance(price, Decimal):
                purchaseTotal = price * Quantity
            else:
                price = Decimal(price)
                purchaseTotal = price * Quantity
        
            productQuery = f"""
            INSERT INTO TRANSACTION_DETAIL (TransactionID, ProductID, Quantity, PurchaseTotal)
            VALUES {(transactionID, productid, Quantity, str(price * Quantity) )}
            """
            cursor.execute(productQuery)
        
        
        connection.commit()
        print("Transaction Created Successfully!")
            
    
            
    except mysql.connector.Error as error: 
        print("Error Creating the Transaction: ", error)
        connection.rollback()
    

In [122]:
# This will get the list of products in product table
ProductListQuery = """
SELECT * 
FROM PRODUCT;
"""

def ProductList():
    try:
        cursor.execute(ProductListQuery)
        
        print("Products: ")
        print(f"{'ProductID':<10} {'Name':<60} {'Price':<10}")
        for product in cursor: 
            print(f"{product[0]:<10} {product[1]:<60} {product[3]:<10}")
        
        connection.commit()
    except mysql.connector.Error as error: 
        print("Error fetching inventory: ", error)
        connection.rollback()


In [123]:
#Console menu function
def mainMenu():
    while True: 
        print("\nConsole Menu")
        print("0. Exit")
        print("1. Products Currently in Inventory.")
        print("2. Create New Product.")
        print("3. Modify a Product in Inventory")
        print("4. Delete a Product from Inventory.")
        print("5. Get Most Popular Products from the given time range.")
        print("6. Get Least Popular Products from the given time range.")
        print("7. Send Promotional Emails to unactive Users.")
        print("8. Create Transaction")
        print("9. Get List of Products")
        
        choice = input("Enter your choice: ")
        match choice: 
            case '0': 
                print("\nExiting.")
                return
            case '1':
                print("\nProducts currently in Inventory are: ")
                InventoryList()
            case '2':
                print("\nAttempting to Add Product.")
                createProduct()
            case '3':
                print("\nEditing Inventory.")
                editInventoryProd()
            case '4': 
                print("Attempting to Deleting Product.")
                deleteProd()
            case '5':
                getMostPop()
            case '6': 
                getLeastPop()
            case '7': 
                print("Getting non Active Users.")
                getAfkUsers()
            case '8': 
                print("Creating Transaction.")
                createNewTransaction()
            case '9':
                print("List of Products: ")
                ProductList()
            case _:
                print("\nPlease enter a digit 0-9.")
            
mainMenu() 



Console Menu
0. Exit
1. Products Currently in Inventory.
2. Create New Product.
3. Modify a Product in Inventory
4. Delete a Product from Inventory.
5. Get Most Popular Products from the given time range.
6. Get Least Popular Products from the given time range.
7. Send Promotional Emails to unactive Users.
8. Create Transaction
9. Get List of Products

Products currently in Inventory are: 
Inventory: 
InventoryID     ProductID       Available      
990000          10000           19             
990001          10001           53             
990002          10002           15             
990003          10003           18             
990004          10004           53             
990005          10005           98             
990006          20000           23             
990007          20001           24             
990008          20002           27             
990009          30000           19             
990010          30001           17             
990011          30

In [72]:
# Closing the connection to the database 

if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

MySQL connection is closed
